In [667]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load generated tsv file from the eurostat website into the dataframe

In [668]:
df_nuts =  pd.read_csv('Input Data/crop_prod_nuts_data.tsv', sep = '\t', header=0, encoding = "ISO-8859-1")

Replace missing values (colons) with NaN

In [669]:
df_nuts = df_nuts.replace(':',np.nan)

* reset indices
* set the first column values as keys

In [670]:
df_nuts.reset_index(drop=True, inplace=True)

In [671]:
df_nuts = df_nuts.set_index('GEO,CROPS,STRUCPRO\TIME')

*Repeat the loading process for the second tsv file*

In [672]:
df_alt= pd.read_csv('Input Data/crop_prod_data.tsv', sep = '\t', header=0, encoding = "ISO-8859-1")

In [673]:
df_alt=df_alt.replace(':',np.nan)

In [674]:
df_alt.reset_index(drop=True, inplace=True)

In [675]:
df_alt=df_alt.set_index('GEO,CROPS,STRUCPRO\TIME')

## Complete missing information with data from the second dataframe

In [676]:
df=df_nuts

Update missing values from the second dataframe

In [677]:
df.update(df_alt, overwrite=False)

In [678]:
df = df.reset_index()

## Clean up and format the table
Split values from the first column into three separate ones

In [679]:
df[['Location','comdty','param']]=df['GEO,CROPS,STRUCPRO\TIME'].str.rsplit(",",2,expand=True)

Move the new columns to the front and drop the original one

In [680]:
df = df[['Location','comdty','param','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']]

Set every NaN value as 0

In [681]:
# df=df.replace(np.nan,0)

Library used to latinize strings

In [682]:
import unicodedata

Function takes string as input <br>
Applies normalize (latinize) method to it <br>
Encodes as ASCII while dropping error characters if any persist <br>
Returns an utf-8 decoded string that has no non-ascii characters

In [683]:
def remove_accents(input_str):
    form=unicodedata.normalize('NFKD',input_str)
    only_ascii=form.encode('ASCII','ignore')
    return only_ascii.decode('utf-8')

Apply prev mentioned function to the Location column

In [684]:
df['Location']=df['Location'].apply(remove_accents)

Renaming Location column to shapeId to adhere to the naming standards

In [685]:
df=df.rename(columns={'Location':'shapeId'})

## Move yearly column values to a single column
 
Splitting dataframe into two along the first encounter of the years column

In [686]:
df_main = df.iloc[:,:3]

In [687]:
df_year = df.iloc[:,3:]

* Create a dictionary of dataframes for each year in the table
* Iterate over every year in the dataframe 
* Concatenate its' column values with the dataframe that consists of param and comdty attributes 
* Rename the 'year' column into 'value' 
* Create a date column and add the corresponding date to it

In [688]:
df_date_merge={}
for (columnName, columnData) in df_year.iteritems():
    df_date_merge[columnName]=pd.concat([df_main,df_year[columnName]],axis=1)
    df_date_merge[columnName]=df_date_merge[columnName].rename(columns={columnName:'value'})
    df_date_merge[columnName].insert(loc=0,column='date',value=columnName+'-01-01 00:00:00')   

Create empty data frame that will house all yearly dataframes <br>
Iterate over each dataframe in the dictionary <br> 
Concatenate it to the final dataframe

In [689]:
df_final=pd.DataFrame()
for key,frame in df_date_merge.items():
    df_final=pd.concat([df_final,frame],ignore_index=True,sort=False)

## Change datatypes of columns
Change **'date'** columns' datatype to **'datetime'** <br>
Remove all non digit elements from the value column except periods ('.')

In [690]:
df_final['date']= pd.to_datetime(df_final['date'])
df_final['value'].replace(regex=True, inplace=True, to_replace=r'[^\d.]', value=r'')

Add the dateRelease column to the table and setting its' type to datetime

In [691]:
df_final.insert(loc=1,column='dateRelease',value="2021-09-23 00:00:00")

In [692]:
df_final['dateRelease']=pd.to_datetime(df_final['dateRelease'])

Set values from the **'value'** column to floats <br>
Set any empty cell to NaN <br>
Empty cells have flags, i.e. "classified", "removed", etc. <br>
**Interpret them as zero values** <br>

In [693]:
df_final['value'] = pd.to_numeric(df_final['value'],errors='coerce')

In [694]:
# df_final['value'] = df_final['value'].replace(np.nan, 0)

Check that all columns have the appropriate type

In [695]:
df_final.dtypes

date           datetime64[ns]
dateRelease    datetime64[ns]
shapeId                object
comdty                 object
param                  object
value                 float64
dtype: object

Writing the data frame to an output csv file

In [696]:
df_final.to_csv('Output Data/crop_prod_eu_stand_humidity_combined.csv')

In [697]:
df_final.to_csv('Output Data/crop_prod_eu_stand_humidity_combined.tsv')

In [698]:
df_final

,date,dateRelease,shapeId,comdty,param,value
0,2012-01-01,2021-09-23,"European Union (EU6-1958, EU9-1973, EU10-1981,...",Wheat and spelt,Area (cultivation/harvested/production) (1000 ha),NaN
1,2012-01-01,2021-09-23,European Union - 27 countries (from 2020),Wheat and spelt,Area (cultivation/harvested/production) (1000 ha),NaN
2,2012-01-01,2021-09-23,European Union - 28 countries (2013-2020),Wheat and spelt,Area (cultivation/harvested/production) (1000 ha),NaN
3,2012-01-01,2021-09-23,Belgium,Wheat and spelt,Area (cultivation/harvested/production) (1000 ha),217.1
4,2012-01-01,2021-09-23,Region de Bruxelles-Capitale/Brussels Hoofdste...,Wheat and spelt,Area (cultivation/harvested/production) (1000 ha),NaN
...,...,...,...,...,...,...
156995,2021-01-01,2021-09-23,"Gaziantep, Adiyaman, Kilis",Durum wheat,EU standard humidity (%),NaN
156996,2021-01-01,2021-09-23,"Sanliurfa, Diyarbakir",Durum wheat,EU standard humidity (%),NaN
156997,2021-01-01,2021-09-23,"Mardin, Batman, Sirnak, Siirt",Durum wheat,EU standard humidity (%),NaN
156998,2021-01-01,2021-09-23,Bosnia and Herzegovina,Durum wheat,EU standard humidity (%),14.0


In [699]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157000 entries, 0 to 156999
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         157000 non-null  datetime64[ns]
 1   dateRelease  157000 non-null  datetime64[ns]
 2   shapeId      157000 non-null  object        
 3   comdty       157000 non-null  object        
 4   param        157000 non-null  object        
 5   value        53751 non-null   float64       
dtypes: datetime64[ns](2), float64(1), object(3)
memory usage: 7.2+ MB


In [700]:
df_final.size

942000

In [701]:
df_final.shape

(157000, 6)